In [52]:
import pypsa
import pandas as pd
import numpy as np
import time
import cufflinks as cf
import plotly.offline as pltly
pltly.init_notebook_mode(connected=True)
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

### Loading network from file

In [440]:
network = pypsa.Network()

In [441]:
network.import_from_hdf5('Scandinavia')

INFO:pypsa.io:Imported network Scandinavia has buses, generators, lines, loads


### Initial solution of network

In [491]:
%%capture 
network.lopf(network.snapshots, solver_name='gurobi')
old_objective_value = network.model.objective()

INFO:pypsa.pf:Slack bus for sub-network 0 is My bus 0
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
Index(['gas 0', 'gas 1', 'gas 2', 'wind 0', 'wind 1', 'wind 2', 'solar 0',
       'solar 1', 'solar 2'],
      dtype='object', name='name')
Currently PyPSA cannot do both these functions, so PyPSA is choosing investment optimisation for these generators.
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


In [492]:
original_solution = []
for j in range(3):
    original_solution.append(sum(network.generators.p_nom_opt[j*3:j*3+3]))

### Finding maximum and minimum points

In [493]:
import pyomo.environ as pyomo_env
# Defining exstra functionality, that updates the objective function of the network
def maxmin_functionality(network, snapshots,  MGA_slack = 0.05, counter = 0):
# Identify the nonzero decision variables that should enter the MGA objective function.

    MGA_objective = 0
    variables = [gen_p for gen_p in network.model.generator_p_nom]
    gen_p1 = variables[int(counter/2)*3]
    gen_p2 = variables[int(counter/2)*3+1]
    gen_p3 = variables[int(counter/2)*3+2]

    MGA_objective += network.model.generator_p_nom[gen_p1]
    MGA_objective += network.model.generator_p_nom[gen_p2]
    MGA_objective += network.model.generator_p_nom[gen_p3]
    if counter%2 == 0 :
        MGA_objective = -MGA_objective
    print(MGA_objective)

    # Add the new MGA objective function to the model.
    network.model.mga_objective = pyomo_env.Objective(expr=MGA_objective)
    # Deactivate the old objective function and activate the MGA objective function.
    network.model.objective.deactivate()
    network.model.mga_objective.activate()
    # Add the MGA slack constraint.
    #print('old objective value ',old_objective_value)
    network.model.mga_constraint = pyomo_env.Constraint(expr=network.model.objective.expr <= 
                                          (1 + MGA_slack) * old_objective_value)

In [494]:
counter = 0
MGA_slack = 0.1

N = 3


for counter in range(6):
    network.lopf(network.snapshots,\
                         solver_name='gurobi',\
                         extra_functionality=lambda network,\
                         snapshots: maxmin_functionality(network, snapshots, MGA_slack,counter))
    network.solutions.append(network.copy())

INFO:pypsa.pf:Slack bus for sub-network 0 is My bus 0
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
Index(['gas 0', 'gas 1', 'gas 2', 'wind 0', 'wind 1', 'wind 2', 'solar 0',
       'solar 1', 'solar 2'],
      dtype='object', name='name')
Currently PyPSA cannot do both these functions, so PyPSA is choosing investment optimisation for these generators.
INFO:pypsa.opf:Solving model using gurobi


- (generator_p_nom[gas 0] + generator_p_nom[gas 1] + generator_p_nom[gas 2])


INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x1813
  Lower bound: -39667.04123703125
  Upper bound: -39667.04123703125
  Number of objectives: 1
  Number of constraints: 3722
  Number of variables: 1813
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 1813
  Number of nonzeros: 8002
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: M

INFO:pypsa.pf:Slack bus for sub-network 0 is My bus 0
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
Index(['gas 0', 'gas 1', 'gas 2', 'wind 0', 'wind 1', 'wind 2', 'solar 0',
       'solar 1', 'solar 2'],
      dtype='object', name='name')
Currently PyPSA cannot do both these functions, so PyPSA is choosing investment optimisation for these generators.
INFO:pypsa.opf:Solving model using gurobi


...

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x1813
  Lower bound: -15302.067050028827
  Upper bound: -15302.067050028827
  Number of objectives: 1
  Number of constraints: 3722
  Number of variables: 1813
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 1813
  Number of nonzeros: 8002
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message:

INFO:pypsa.pf:Slack bus for sub-network 0 is My bus 0
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
Index(['gas 0', 'gas 1', 'gas 2', 'wind 0', 'wind 1', 'wind 2', 'solar 0',
       'solar 1', 'solar 2'],
      dtype='object', name='name')
Currently PyPSA cannot do both these functions, so PyPSA is choosing investment optimisation for these generators.
INFO:pypsa.opf:Solving model using gurobi


generator_p_nom[solar 0] + generator_p_nom[solar 1] + generator_p_nom[solar 2]


INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x1813
  Lower bound: 0.0
  Upper bound: 0.0
  Number of objectives: 1
  Number of constraints: 3722
  Number of variables: 1813
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 1813
  Number of nonzeros: 8002
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality 

In [495]:
max_min_points = []
for i in range(len(network.solutions)):
    test = []
    for j in range(3):
        test.append(sum(network.solutions[i].generators.p_nom_opt[j*3:j*3+3]))
    max_min_points.append(test)

### Plot of max and min points 

In [447]:
%matplotlib widget
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
  
for i in range(len(max_min_points)):
    ax.plot([max_min_points[i][0]],[max_min_points[i][1]],[max_min_points[i][2]], "go")

ax.set_xlabel('gas')
ax.set_ylabel('wind')
ax.set_zlabel('solar')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'solar')

### Defining functions to test individual points 

In [496]:
def constraint_functionality(network, snapshots,  MGA_slack = 0.05, constraint_values = [-1,-1,-1]):
    import pyomo.environ as pyomo_env

    variables = [gen_p for gen_p in network.model.generator_p_nom]
    
    if constraint_values[0] >= 0 :
        gas_constraint = network.model.generator_p_nom[variables[0]]
        gas_constraint += network.model.generator_p_nom[variables[1]]
        gas_constraint += network.model.generator_p_nom[variables[2]]
        network.model.gas_constraint = pyomo_env.Constraint(expr=gas_constraint==constraint_values[0])

    if constraint_values[1] >= 0 :
        wind_constraint = network.model.generator_p_nom[variables[3]]
        wind_constraint += network.model.generator_p_nom[variables[4]]
        wind_constraint += network.model.generator_p_nom[variables[5]]
        network.model.wind_constraint = pyomo_env.Constraint(expr=wind_constraint==constraint_values[1])

    if constraint_values[2] >= 0 :
        solar_constraint = network.model.generator_p_nom[variables[6]]
        solar_constraint += network.model.generator_p_nom[variables[7]]
        solar_constraint += network.model.generator_p_nom[variables[8]]
        network.model.solar_constraint = pyomo_env.Constraint(expr=solar_constraint==constraint_values[2])


In [449]:
def test_point(network,variables):

    out = network.lopf(network.snapshots,\
                 solver_name='gurobi',\
                 skip_pre=True,\
                 extra_functionality=lambda network,\
                 snapshots: constraint_functionality(network, snapshots, MGA_slack,variables),
                 solver_options = {'threads':8});

    feasable = str(network.results['Solver'].status) == 'ok' and (network.model.objective()<= (1 + MGA_slack) * old_objective_value)
      
    variables_out = [sum(network.generators.p_nom_opt[0:3]),sum(network.generators.p_nom_opt[3:6]),sum(network.generators.p_nom_opt[6:9])]

    return(feasable,variables_out)

In [450]:
def isBetween(a, b, c):
    # a Feasable point 
    # b infeasable point or feasable point if between test
    # c point to be tested
    
    between = True
    outside = False
    
    
    crossproduct = np.cross((b-a),(c-a))
    
    dotproduct = np.dot((c-a),(b-a))
                                
    # compare versus epsilon for floating point values, or != 0 if using integers
    # Test if c is on same axis as a and b
    #print(abs(crossproduct))
    epsilon = np.sqrt(abs(dotproduct))/100
    #print(epsilon)
    if any(abs(crossproduct) > epsilon):
        # c not in line with a and b
        between =  False
        outside = False
        return between, outside
    
    if all(a == b):
        between = False
        outside = False
        return between, outside      

    squaredlengthba = np.linalg.norm(b-a)**2
    #print(squaredlengthba)
    if dotproduct > squaredlengthba:
        between = False
        outside = False
        return between, outside
    
    #print(dotproduct)
    # Test if point c is further away from a then point b
    if dotproduct < 0:
        between =  False
        outside = True 
        return between, outside

    return between, outside
    

### Creating grid of points  

In [472]:
class grid :
    pass

dens = 11
grid.gas, grid.wind,grid.solar  = np.meshgrid(np.linspace(max_min_points[1][0],max_min_points[0][0],dens),\
                                            np.linspace(max_min_points[3][1],max_min_points[2][1],dens),\
                                            np.linspace(max_min_points[5][2],max_min_points[4][2],dens))
grid.gas = grid.gas.reshape((dens)**3)
grid.wind = grid.wind.reshape((dens)**3)
grid.solar = grid.solar.reshape((dens)**3)

mid_point = [grid.gas.mean(),grid.wind.mean(),grid.solar.mean()]
dist_to_mid=np.sqrt(abs(grid.gas - mid_point[0])**2+abs(grid.wind - mid_point[1])**2+abs(grid.solar - mid_point[2])**2)
sort_idx = np.argsort(dist_to_mid)
grid.gas = grid.gas[sort_idx]
grid.wind = grid.wind[sort_idx]
grid.solar = grid.solar[sort_idx]

In [473]:
print('Computation time estimate: ' + str(len([[x,y,z] for x,y,z in zip(grid.gas,grid.solar,grid.wind)])*1.3/60) + 'min')

Computation time estimate: 28.83833333333333min


### Computing feasable points

In [474]:
%%capture
t = time.time()

grid.feasable = []
grid.infeasable = []
grid.irelevant = []
simulations = 0

for gas,wind,solar in zip(grid.gas,grid.wind,grid.solar):
    
    point3 = np.array([gas,wind,solar])
    
    between = any([isBetween(point1,point2,point3)[0] for point1 in grid.feasable for point2 in grid.feasable])
    outside = any([isBetween(point2,point1,point3)[1] for point1 in grid.feasable for point2 in grid.infeasable])
    
    if (not between) and (not outside) :
        is_good , _ = test_point(network,[gas,wind,solar])
        simulations += 1
        if is_good:
            grid.feasable.append(point3)
        else : 
            grid.infeasable.append(point3)
    else outside :
        grid.irelevant.append(point3)
        
elapsed = time.time() - t


INFO:pypsa.opf:Building pyomo model using `angles` formulation
Index(['gas 0', 'gas 1', 'gas 2', 'wind 0', 'wind 1', 'wind 2', 'solar 0',
       'solar 1', 'solar 2'],
      dtype='object', name='name')
Currently PyPSA cannot do both these functions, so PyPSA is choosing investment optimisation for these generators.
INFO:pypsa.opf:Solving model using gurobi


3662563885.825492
 gas constraint
generator_p_nom[gas 0] + generator_p_nom[gas 1] + generator_p_nom[gas 2]  ==  34688.38664585003
 wind constraint
generator_p_nom[wind 0] + generator_p_nom[wind 1] + generator_p_nom[wind 2]  ==  3665.490702150364
 solar constraint
generator_p_nom[solar 0] + generator_p_nom[solar 1] + generator_p_nom[solar 2]  ==  7651.0335250144135


INFO:pypsa.opf:Optimization successful
INFO:pypsa.opf:Building pyomo model using `angles` formulation
Index(['gas 0', 'gas 1', 'gas 2', 'wind 0', 'wind 1', 'wind 2', 'solar 0',
       'solar 1', 'solar 2'],
      dtype='object', name='name')
Currently PyPSA cannot do both these functions, so PyPSA is choosing investment optimisation for these generators.


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x1813
  Lower bound: 3831399962.4198694
  Upper bound: 3831399962.4198694
  Number of objectives: 1
  Number of constraints: 3724
  Number of variables: 1813
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 1813
  Number of nonzeros: 7639
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: M

INFO:pypsa.opf:Solving model using gurobi


...

    model=Linear Optimal Power Flow;
        message from solver=Problem proven to be infeasible or unbounded.
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x1813
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 3724
  Number of variables: 1813
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 1813
  Number of nonzeros: 7639
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: warning
  Return code: 0
  Message: Problem proven to be infeasible or unbounded.
  Termination cond

INFO:pypsa.opf:Building pyomo model using `angles` formulation
Index(['gas 0', 'gas 1', 'gas 2', 'wind 0', 'wind 1', 'wind 2', 'solar 0',
       'solar 1', 'solar 2'],
      dtype='object', name='name')
Currently PyPSA cannot do both these functions, so PyPSA is choosing investment optimisation for these generators.
INFO:pypsa.opf:Solving model using gurobi


3662563885.825492
 gas constraint
generator_p_nom[gas 0] + generator_p_nom[gas 1] + generator_p_nom[gas 2]  ==  39667.04123703125
 wind constraint
generator_p_nom[wind 0] + generator_p_nom[wind 1] + generator_p_nom[wind 2]  ==  0.0
 solar constraint
generator_p_nom[solar 0] + generator_p_nom[solar 1] + generator_p_nom[solar 2]  ==  0.0


INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x1813
  Lower bound: 3662563885.8254924
  Upper bound: 3662563885.8254924
  Number of objectives: 1
  Number of constraints: 3724
  Number of variables: 1813
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 1813
  Number of nonzeros: 7639
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: M

In [476]:
print(elapsed/60)
print(simulations)

42.01154630184173
398
6.333398939976141


In [478]:
grid.feasable = np.array(grid.feasable)
grid.infeasable = np.array(grid.infeasable)
grid.irelevant = np.array(grid.irelevant)

In [486]:
np.save('feasable_points',grid.feasable,max_min_points)

In [480]:
#grid.feasable = np.load('feasable_points.npy')

### Plotting feasable region

In [485]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for i in range(len(max_min_points)):
       ax.plot([max_min_points[i][0]],[max_min_points[i][1]],[max_min_points[i][2]], "go",markersize=10)


ax.plot(grid.feasable[:,0],grid.feasable[:,1],grid.feasable[:,2], "bo",markersize=7)
#ax.plot(grid.infeasable[:,0],grid.infeasable[:,1],grid.infeasable[:,2], "ro",markersize=7)
#ax.plot(grid.irelevant[:,0],grid.irelevant_outside[:,1],grid.irelevant_outside[:,2], "ko",markersize=6)
ax.plot([original_solution[0]],[original_solution[1]],[original_solution[2]],'yo')
    
plt.show();
ax.set_xlabel('Gas');
ax.set_ylabel('Wind');
ax.set_zlabel('Solar');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Solar')